In [120]:
from faker import Faker
from random import randint, choice, uniform
from datetime import datetime, timedelta
import pandas as pd

# Initialize Faker
fake = Faker()

# Generate a list of trader IDs and companies
trader_ids = [fake.random_number(digits=5) for _ in range(10)]  # 10 unique trader IDs
companies = ["JP Morgan", "Morgan Stanley", "Deutsche Bank", "Goldman Sachs", "BNY"]  # companies
    
# Define the structure of your transaction data
def generate_single_transaction():


    # Define a list of currency pairs and corresponding spot rate range
    currency_data = {
        'EUR/USD': {'spot_rate_range': (1.1, 1.2)},
        'JPY/USD': {'spot_rate_range': (1.0 / 115.0, 1.0 / 105)},
        'GBP/USD': {'spot_rate_range': (1.3, 1.4)},
        'CHF/USD': {'spot_rate_range': (1.0, 1.1)},
        'CAD/USD': {'spot_rate_range': (0.8, 1.0)},
        'AUD/USD': {'spot_rate_range': (0.7, 0.8)},
        'NZD/USD': {'spot_rate_range': (0.6, 0.7)},
    }

    currency_pair = choice(list(currency_data.keys()))
    buy_currency, sell_currency = currency_pair.split('/')
    rate_data = currency_data[currency_pair]
    buy_sell_indicator = choice(['Buy', 'Sell'])
    quantity = randint(1, 1000) * 10000
    spot_rate = round(fake.pyfloat(right_digits=4, positive=True, min_value=rate_data['spot_rate_range'][0], max_value=rate_data['spot_rate_range'][1]), 4)  # generating a spot rate within the range
    # calculate a forward rate within 1% of the spot rate
    forward_rate = round(spot_rate * uniform(0.99, 1.01), 4)  
    return {
        'TransactionID': str(fake.unique.random_number(digits=8)),
        'TradingDate': fake.date_between(start_date='-1y', end_date='today'),
        'MaturityDate': fake.date_between(start_date='today', end_date='+3M'),
        'ExecutionTime': (datetime.now()-timedelta(seconds=randint(1,86400))).strftime('%H:%M:%S'),
        'InstrumentID': fake.random_number(digits=12),
        'TraderID': choice(trader_ids),  # select a trader ID from the list
        'BuyCurrency': buy_currency,
        'SellCurrency': sell_currency,
        'SpotRate': spot_rate,
        'ForwardRate': forward_rate,
        'BuySellIndicator': buy_sell_indicator,
        'BuyNotional': round(quantity, 2),
        'SellNotional': round(quantity * spot_rate, 2),
        'Counterparty': choice(companies)  # select a company from the list
    }

# Generate n transactions
def generate_transactions(n=100):
    return pd.DataFrame([generate_single_transaction() for _ in range(n)])

transactions = generate_transactions(1000)

def portfolio():
    return transactions

In [164]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from scipy.stats import norm

# Assume transaction_df is already provided
# Define a time grid
time_grid = list(range(1, 15)) + list(range(14, 91, 7)) + list(range(90, 366, 30))

# Mock functions to fetch volatility and correlation matrix
def fetch_vol_and_correlation(alternative=False):
    volatilities_usd = {
        'EUR': 0.1,
        'JPY': 0.15,
        'GBP': 0.12,
        'CHF': 0.1,
        'CAD': 0.13,
        'AUD': 0.14,
        'NZD': 0.15
    }

    correlation_matrix_usd = np.array([
        [1.0, 0.3, 0.2, 0.1, 0.1, 0.2, 0.1],
        [0.3, 1.0, 0.4, 0.2, 0.2, 0.1, 0.2],
        [0.2, 0.4, 1.0, 0.3, 0.3, 0.2, 0.3],
        [0.1, 0.2, 0.3, 1.0, 0.4, 0.3, 0.2],
        [0.1, 0.2, 0.3, 0.4, 1.0, 0.2, 0.1],
        [0.2, 0.1, 0.2, 0.3, 0.2, 1.0, 0.4],
        [0.1, 0.2, 0.3, 0.2, 0.1, 0.4, 1.0]
    ])
    return volatilities_usd, correlation_matrix_usd

# Functions to calculate PFE
def convert_to_usd(notional, currency, spot_rate):
    if currency == 'USD':
        return notional
    return notional * spot_rate

# Create an exposure vector
def create_exposure_vector(buy_currency, sell_currency, buy_notional_usd, sell_notional_usd, volatilities_usd):
    currencies = list(volatilities_usd.keys())
    exposures = np.zeros(len(currencies))

    for i, currency in enumerate(currencies):
        if currency == buy_currency:
            exposures[i] += buy_notional_usd
        if currency == sell_currency:
            exposures[i] -= sell_notional_usd
    return exposures

# Create an exposure vector
def create_aggregated_exposure_vector(aggregated_exposures, volatilities_usd):
    currencies = list(volatilities_usd.keys())
    exposures = np.zeros(len(currencies))

    for currency, notional in aggregated_exposures.items():
        if currency in currencies:
            index = currencies.index(currency)
            exposures[index] = notional
    return exposures


def calculate_pfe_with_cov_matrix(exposures, days, cov_matrix, confidence_level=0.99):
    # ensure that the length of exposure is equal to a side of the covariance matrix
    if len(exposures) != cov_matrix.shape[0]:
        raise ValueError("Length of exposures should match the covariance matrix")
    
    exposures_vector = np.array(exposures)
    # adjust the covariance matrix for the number of days, the covariance matrix is assumed to be annualized
    cov_matrix = cov_matrix * days / 365
    
    variance = exposures_vector.T @ cov_matrix @ exposures_vector
    stddev = np.sqrt(variance)
    z_score = norm.ppf(confidence_level)
    
    pfe = stddev * z_score
    return pfe


def calculate_pfe_for_trade(row, volatilities, cov_matrix, time_grid, mpor):
    pfe_results = {}
    buy_currency = row['BuyCurrency']
    sell_currency = row['SellCurrency']
    spot_rate = row['SpotRate']
    buy_notional_usd = convert_to_usd(row['BuyNotional'], buy_currency, spot_rate)
    sell_notional_usd = convert_to_usd(row['SellNotional'], sell_currency, spot_rate)
    
    for days in time_grid:
        if row['MaturityDate'] < row['TradingDate'] + timedelta(days=days):
            continue  # Exclude matured trades

        exposures = create_exposure_vector(buy_currency, sell_currency, buy_notional_usd, sell_notional_usd, volatilities)
        
        standalone_pfe = calculate_pfe_with_cov_matrix(exposures, days, cov_matrix, confidence_level=0.99)
        collateralized_pfe = calculate_pfe_with_cov_matrix(exposures, mpor, cov_matrix, confidence_level=0.99)
        
        pfe_results[days] = {
            'Standalone': standalone_pfe,
            'Collateralized': collateralized_pfe
        }
    
    return pfe_results

def aggregate_exposures_by_counterparty(transactions_df, time):
    counterparty_exposures = {}
    
    for _, row in transactions_df.iterrows():
        
        # Skip matured trades
        if row['MaturityDate'] < row['TradingDate'] + timedelta(days=time):
            continue
        
        buy_currency = row['BuyCurrency']
        sell_currency = row['SellCurrency']
        spot_rate = row['SpotRate']
        buy_notional_usd = convert_to_usd(row['BuyNotional'], buy_currency, spot_rate)
        sell_notional_usd = convert_to_usd(row['SellNotional'], sell_currency, spot_rate)
        counterparty = row['Counterparty']
        
        if counterparty not in counterparty_exposures:
            counterparty_exposures[counterparty] = {}
        
        if buy_currency not in counterparty_exposures[counterparty]:
            counterparty_exposures[counterparty][buy_currency] = 0
        if sell_currency not in counterparty_exposures[counterparty]:
            counterparty_exposures[counterparty][sell_currency] = 0
            
        counterparty_exposures[counterparty][buy_currency] += buy_notional_usd
        counterparty_exposures[counterparty][sell_currency] -= sell_notional_usd
    
    return counterparty_exposures

def calculate_pfe_for_counterparties(transactions_df, volatilities, cov_matrix, time_grid, mpor):
    counterparty_pfe = []
    
    for days in time_grid:
        agg_exp = aggregate_exposures_by_counterparty(transactions_df, days)
        
        # iterate over each counterparty and calculate PFE
        for counterparty, exposures in agg_exp.items():
            aggregated_exposures = create_aggregated_exposure_vector(exposures, volatilities)
            uncol_pfe = calculate_pfe_with_cov_matrix(aggregated_exposures, days, cov_matrix, confidence_level=0.99)
            col_pfe = calculate_pfe_with_cov_matrix(aggregated_exposures, mpor, cov_matrix, confidence_level=0.99)
            
            counterparty_pfe.append(
                    {
                        'Counterparty': counterparty,
                        'Days': days,
                        'UncolPFE': uncol_pfe,
                        'ColPFE': col_pfe
                    }
                )
    return pd.DataFrame(counterparty_pfe)

def CalculatePFE(transactions_df):
    # Margin period of risk (MPOR) in days
    mpor = 10

    # Fetch volatilities and correlation matrix
    volatilities_usd, correlation_matrix_usd = fetch_vol_and_correlation()

    # Calculate variance-covariance matrix for USD pairs
    volatility_vector = np.array(list(volatilities_usd.values()))
    cov_matrix_usd = np.dot(volatility_vector[:, None], volatility_vector[None, :]) * correlation_matrix_usd

    # Calculate PFE for each counterparty
    counterparty_pfe_df = calculate_pfe_for_counterparties(transactions_df, volatilities_usd, cov_matrix_usd, time_grid, mpor)

    # Go thru each transaction_df and calculate PFE as a full term structure in a new dataframe, with the trade id as the key
    pfe_results = []
    for idx, row in transactions_df.iterrows():
        trade_pfe = calculate_pfe_for_trade(row, volatilities_usd, cov_matrix_usd, time_grid, mpor)
        # Append the trade ID and cpty to pfe_results to the trade_pfe, no need to flatten the dictionary
        pfe_results.append(
            {
                'TransactionID': row['TransactionID'], 
                'Counterparty': row['Counterparty'], 
                'BuyCurrency': row['BuyCurrency'],
                'SellCurrency': row['SellCurrency'],
                'MaturityDate': row['MaturityDate'],
                'PFE': trade_pfe})
        
    # flatten pfe_results and PFE with the days as a new column to a dataframe
    pfe_results_df = []
    for row in pfe_results:
        for days, pfe in row['PFE'].items():
            pfe_results_df.append(
                {'TransactionID': row['TransactionID'], 
                'Counterparty': row['Counterparty'], 
                'BuyCurrency': row['BuyCurrency'],
                'SellCurrency': row['SellCurrency'],
                'MaturityDate': row['MaturityDate'],
                'Days': days, 
                'Standalone_PFE': pfe['Standalone'], 
                'Collateralized_PFE': pfe['Collateralized']})

    pfe_results_df = pd.DataFrame(pfe_results_df)
    
    return counterparty_pfe_df, pfe_results_df

def UniqueCounterparties(transactions_df):
    return transactions_df['Counterparty'].unique()

def ComputeMarkedToMarket(transactions_df, scenarios=None):
    # Go thru each transaction and calculate the marked to market for each scenario including the base scenario (no change in rates)
    marked_to_market = []
    
    if scenarios is None:
        # Define a base scenario of no change in rates
        scenarios = [
            {
                'Name': 'Base',
                'Impact': {}
            }
        ]

    for idx, row in transactions_df.iterrows():
        for scenario in [None] + scenarios:
            if scenario is None:
                # Base scenario
                forward_rate = row['ForwardRate']
            else:
                # Apply scenario impact to both buy and sell currencies
                buy_impact = scenario['Impact'].get(row['BuyCurrency'], 0)
                sell_impact = scenario['Impact'].get(row['SellCurrency'], 0)
                forward_rate = row['ForwardRate'] * (1 + buy_impact - sell_impact)

            # Calculate marked to market
            if row['BuySellIndicator'] == 'Buy':
                mtm = row['BuyNotional'] * forward_rate - row['SellNotional'] 
            else:
                mtm = row['SellNotional'] -  row['BuyNotional'] * forward_rate
            
            marked_to_market.append(
                {
                    'TransactionID': row['TransactionID'],
                    'Counterparty': row['Counterparty'], 
                    'BuyCurrency': row['BuyCurrency'], 
                    'SellCurrency': row['SellCurrency'],
                    'MaturityDate': row['MaturityDate'],
                    'Scenario': scenario['Name'] if scenario is not None else 'Base',
                    'MarkedToMarket': mtm
                }
            )            
    return pd.DataFrame(marked_to_market)
            

def DefineScenarios():
    # Define a list of scenarios for the stress testing in the FX defined EUR, JPY, GBP, CHF, CAD, AUD, NZD
    
    scenarios = [
        {
            'Name': 'Brexit',
            # Impact on each currency in the list defined EUR, JPY, GBP, CHF, CAD, AUD, NZD
            'Impact': {
                'EUR' : -0.05,
                'GBP' : -0.1,
                'JPY' : 0.05,
                'CHF' : 0.05,
                'CAD' : 0.05,
                'AUD' : 0.05,
                'NZD' : 0.05
            }
        },
        {
            'Name': 'US-China Trade War',
            'Impact': {
                'JPY' : 0.1,
                'EUR' : -0.05,
                'GBP' : -0.05,
                'CHF' : -0.05,
                'CAD' : -0.05,
                'AUD' : -0.05,
                'NZD' : -0.05
            }
        },
        {
            'Name': 'COVID-19',
            'Impact': {
                'JPY' : 0.2,
                'EUR' : 0.07,
                'GBP' : 0.1,
                'CHF' : 0.1,
                'CAD' : 0.07,
                'AUD' : 0.1,
                'NZD' : 0.1
            }
        }
    ]
        
    return scenarios

def ComputeMaxLossByCounterparty(transactions_df, scenarios=None):
    # Group by Counterparty and calculate the sum of scenarios
    mtm_df = ComputeMarkedToMarket(transactions_df, DefineScenarios())
    marked_to_market_df = mtm_df.groupby(['Counterparty', 'Scenario']).sum()
    # Create a new column and subtract the base scenario from the other scenarios
    marked_to_market_df['MarkedToMarket_Diff'] = marked_to_market_df.groupby('Counterparty')['MarkedToMarket'].diff()
    # Drop the base scenario
    marked_to_market_df.dropna(inplace=True)
    return marked_to_market_df

In [165]:
counterparty_pfe_df, pfe_results_df = CalculatePFE(transactions)
ComputeMaxLossByCounterparty(transactions, DefineScenarios())

/var/folders/jv/hd_kl00j2hxfm6sn1phxsr1m0000gn/T/ipykernel_35324/412691332.py:296: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  marked_to_market_df = mtm_df.groupby(['Counterparty', 'Scenario']).sum()


MarkedToMarket  MarkedToMarket_Diff
Counterparty   Scenario                                               
BNY            Brexit                 -7115015.30          -7290225.30
               COVID-19               -4549436.16           2565579.14
               US-China Trade War      3230021.45           7779457.61
Deutsche Bank  Brexit                 -6069641.10          -5906586.10
               COVID-19               -2175289.63           3894351.47
               US-China Trade War       702925.95           2878215.58
Goldman Sachs  Brexit                  5328591.70           5060594.70
               COVID-19               -6688186.96         -12016778.66
               US-China Trade War      3786505.65          10474692.61
JP Morgan      Brexit                  -302831.80             90407.20
               COVID-19                6222888.19           6525719.99
               US-China Trade War     -4085753.65         -10308641.84
Morgan Stanley Brexit                   991575.65           1979610.65
               COVID-19               16474304.19          15482728.54
               US-China Trade War    -10841500.45         -27315804.64

In [188]:
def maxloss() -> str:
    return ComputeMaxLossByCounterparty(transactions, DefineScenarios()).to_json()

def cpty_exp() -> str:
    return str(aggregate_exposures_by_counterparty(transactions, 0))

def scenarios() -> str:
    return str(DefineScenarios())

def cpty_pfe() -> str:
    counterparty_pfe_df, pfe_results_df = CalculatePFE(transactions)
    return counterparty_pfe_df.to_json()

In [231]:
import os
from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "Here's some tips for resolving issues:"
    "- If you have been asked to calculate how much more room you have for a particular trade, you need to retreive the scenarios then work out the notional with loss being notional x stress scenario"
    "- There is a difference between notional and loss, make sure that you get it right"
    "Now some general tips:"
    "Output the result in nice printable format with thousands separators where appropiate"
    "Return 'TERMINATE' when the task is done."
    ,
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(name="maxloss", description="Calculate the maximum loss by stress scenarios all the numbers are in USD")(maxloss)
assistant.register_for_llm(name="cpty_exp", description="Aggregate exposures by counterparty by currency in USD equivalent, so all numbers are in USD by default")(cpty_exp)
assistant.register_for_llm(name="scenarios", description="Define stress scenarios for FX as decimals")(scenarios)
assistant.register_for_llm(name="cpty_pfe", description="Calculate PFE by counterparty by days in futures and in USD")(cpty_pfe)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(name="maxloss")(maxloss)
user_proxy.register_for_execution(name="cpty_exp")(cpty_exp)
user_proxy.register_for_execution(name="scenarios")(scenarios)
user_proxy.register_for_execution(name="cpty_pfe")(cpty_pfe)

<function __main__.cpty_pfe() -> str>

In [232]:
chatresult = user_proxy.initiate_chat(assistant, 
                                      message="what is the biggest ccy exposure for Bank of New York? Also display other currency exposures." 
                                                "Also tell me the stress loss for this portfolio, now do the same for JPM and let me know the key differences")

User (to Assistant):

what is the biggest ccy exposure for Bank of New York? Also display other currency exposures.Also tell me the stress loss for this portfolio, now do the same for JPM and let me know the key differences

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_mjfFEORuKFGsbjngSU9V3tNa): cpty_exp *****
Arguments: 
{}
*************************************************************************
***** Suggested tool call (call_A7rcmHdjv0JffMH7gE0ELrhz): maxloss *****
Arguments: 
{}
************************************************************************
***** Suggested tool call (call_tteeVdIUDB7FmMSww8Sa7q4O): scenarios *****
Arguments: 
{}
**************************************************************************
***** Suggested tool call (call_sFB3sKgKTagsAq5fBgSLPLpB): cpty_exp *****
Arguments: 
{}
****************************************************************

/var/folders/jv/hd_kl00j2hxfm6sn1phxsr1m0000gn/T/ipykernel_35324/412691332.py:296: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  marked_to_market_df = mtm_df.groupby(['Counterparty', 'Scenario']).sum()



>>>>>>>> EXECUTING FUNCTION scenarios...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_mjfFEORuKFGsbjngSU9V3tNa) *****
{'Morgan Stanley': {'EUR': 176023211.0, 'USD': -804488158.0, 'NZD': 85969715.0, 'CHF': 126589391.0, 'GBP': 173819591.0, 'CAD': 142867921.0, 'JPY': 1333835.0, 'AUD': 97884494.0}, 'Goldman Sachs': {'GBP': 268706852.0, 'USD': -938662424.0, 'CAD': 120881322.0, 'NZD': 105984132.0, 'EUR': 172605469.0, 'JPY': 1789805.0, 'AUD': 103432467.0, 'CHF': 165262377.0}, 'Deutsche Bank': {'CHF': 163009063.0, 'USD': -813888182.0, 'CAD': 108744181.0, 'NZD': 85125732.0, 'EUR': 176199668.0, 'JPY': 1189289.0, 'GBP': 192401812.0, 'AUD': 87218437.0}, 'JP Morgan': {'CAD': 153576400.0, 'USD': -803053227.0, 'CHF': 156796989.0, 'AUD': 107909045.0, 'EUR': 188885678.0, 'JPY': 780202.0, 'GBP': 112039152.0, 'NZD': 83065761.0}, 'BNY': {'JPY': 1259201.0, 'USD': -876372453.0, 'NZD': 89451186.0, 'CAD': 152235653.0, 'CHF': 197358963.0, 'AUD': 96977260.0, 'GBP': 2505632

/var/folders/jv/hd_kl00j2hxfm6sn1phxsr1m0000gn/T/ipykernel_35324/412691332.py:296: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  marked_to_market_df = mtm_df.groupby(['Counterparty', 'Scenario']).sum()


Assistant (to User):

## BNY Exposures

**Currency Exposures (USD Equivalent):**
- **JPY:** 1,259,201 
- **USD:** -876,372,453 
- **NZD:** 89,451,186 
- **CAD:** 152,235,653 
- **CHF:** 197,358,963 
- **AUD:** 96,977,260 
- **GBP:** 250,563,230 
- **EUR:** 88,526,960 

**Biggest Currency Exposure:** **GBP** (250,563,230 USD)

**Stress Losses (USD):**
- **Brexit:** -7,115,015 
- **COVID-19:** -4,549,436 
- **US-China Trade War:** 3,230,021 

---

## JP Morgan Exposures

**Currency Exposures (USD Equivalent):**
- **JPY:** 780,202 
- **USD:** -803,053,227 
- **NZD:** 83,065,761 
- **CAD:** 153,576,400 
- **CHF:** 156,796,989 
- **AUD:** 107,909,045 
- **GBP:** 112,039,152 
- **EUR:** 188,885,678 

**Biggest Currency Exposure:** **EUR** (188,885,678 USD)

**Stress Losses (USD):**
- **Brexit:** -302,832
- **COVID-19:** 6,222,888
- **US-China Trade War:** -4,085,754

---


## Key Differences

**Biggest Currency Exposure:**
- **BNY:** GBP (250,563,230 USD)
- **JP Morgan:** EUR (188,885,678 US

In [233]:
print(chatresult.summary)

## BNY Exposures

**Currency Exposures (USD Equivalent):**
- **JPY:** 1,259,201 
- **USD:** -876,372,453 
- **NZD:** 89,451,186 
- **CAD:** 152,235,653 
- **CHF:** 197,358,963 
- **AUD:** 96,977,260 
- **GBP:** 250,563,230 
- **EUR:** 88,526,960 

**Biggest Currency Exposure:** **GBP** (250,563,230 USD)

**Stress Losses (USD):**
- **Brexit:** -7,115,015 
- **COVID-19:** -4,549,436 
- **US-China Trade War:** 3,230,021 

---

## JP Morgan Exposures

**Currency Exposures (USD Equivalent):**
- **JPY:** 780,202 
- **USD:** -803,053,227 
- **NZD:** 83,065,761 
- **CAD:** 153,576,400 
- **CHF:** 156,796,989 
- **AUD:** 107,909,045 
- **GBP:** 112,039,152 
- **EUR:** 188,885,678 

**Biggest Currency Exposure:** **EUR** (188,885,678 USD)

**Stress Losses (USD):**
- **Brexit:** -302,832
- **COVID-19:** 6,222,888
- **US-China Trade War:** -4,085,754

---


## Key Differences

**Biggest Currency Exposure:**
- **BNY:** GBP (250,563,230 USD)
- **JP Morgan:** EUR (188,885,678 USD)

**Stress Scenarios

In [234]:
chatresult = user_proxy.initiate_chat(assistant, 
                                      message="What is the stress loss for JPM and given the stress scenario, how much more buy USD/GBP can we do if the lmit is 20M USD?")

User (to Assistant):

What is the stress loss for JPM and given the stress scenario, how much more buy USD/GBP can we do if the lmit is 20M USD?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_48fFhRzWT0IWJkGSlDyNgU8t): maxloss *****
Arguments: 
{}
************************************************************************
***** Suggested tool call (call_APB4EeTeItRq0lBntES6BSzM): cpty_exp *****
Arguments: 
{}
*************************************************************************
***** Suggested tool call (call_kSjmWSRvAz6MsNLLHBlJH8gd): scenarios *****
Arguments: 
{}
**************************************************************************
***** Suggested tool call (call_32lWcKSffoXaWIRuAg0aHuNH): cpty_pfe *****
Arguments: 
{}
*************************************************************************

--------------------------------------------------------------------

/var/folders/jv/hd_kl00j2hxfm6sn1phxsr1m0000gn/T/ipykernel_35324/412691332.py:296: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  marked_to_market_df = mtm_df.groupby(['Counterparty', 'Scenario']).sum()


User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_48fFhRzWT0IWJkGSlDyNgU8t) *****
{"MarkedToMarket":{"('BNY', 'Brexit')":-7115015.300000011,"('BNY', 'COVID-19')":-4549436.1600000011,"('BNY', 'US-China Trade War')":3230021.450000009,"('Deutsche Bank', 'Brexit')":-6069641.0999999912,"('Deutsche Bank', 'COVID-19')":-2175289.630000012,"('Deutsche Bank', 'US-China Trade War')":702925.9499999987,"('Goldman Sachs', 'Brexit')":5328591.7000000151,"('Goldman Sachs', 'COVID-19')":-6688186.9599999897,"('Goldman Sachs', 'US-China Trade War')":3786505.6500000074,"('JP Morgan', 'Brexit')":-302831.8000000003,"('JP Morgan', 'COVID-19')":6222888.1899999967,"('JP Morgan', 'US-China Trade War')":-4085753.6500000027,"('Morgan Stanley', 'Brexit')":991575.6500000115,"('Morgan Stanley', 'COVID-19')":16474304.1900000162,"('Morgan Stanley', 'US-China Trade War')":-10841500.450000003},"MarkedToMarket_Diff":{"('BNY', 'Brexit')":-7290225.300000011,"('BNY', 'COVID-19')":2565579.1400

In [235]:
print(chatresult.summary)

To calculate the stress loss for JP Morgan and determine how much more USD/GBP can be traded if the limit is 20M USD, given the stress scenarios, follow these steps:

1. **Retrieve JP Morgan's Exposures and corresponding losses**:
    - GBP Exposure for JP Morgan: \( 112,039,152 \, \text{USD} \)
    - Stress Scenarios:
        - Brexit: \( -0.1 \)
        - US-China Trade War: \( -0.05 \)
        - COVID-19: \( 0.1 \)
    - Current Exposure variations (losses) for JP Morgan:
        - Brexit: \( -302,832 \, \text{USD} \)
        - US-China Trade War: \( -4,085,754 \, \text{USD} \)
        - COVID-19: \( 6,222,888 \, \text{USD} \)

2. **Calculate potential losses under stress scenarios for JP Morgan:**
    - Brexit Stress Loss: \( 112,039,152 \times -0.1 = -11,203,915.2 \, \text{USD} \)
    - US-China Trade War Stress Loss: \( 112,039,152 \times -0.05 = -5,601,957.6 \, \text{USD} \)
    - COVID-19 Stress Loss: \( 112,039,152 \times 0.1 = 11,203,915.2 \, \text{USD} \)

3. **Identify the 

In [236]:
chatresult = user_proxy.initiate_chat(assistant, 
                                      message="How much more can we buy in USD/GBP in notional can we do if the stress loss limit is 20M USD per cpty, can you work out the room in each cpties then total it?")

User (to Assistant):

How much more can we buy in USD/GBP in notional can we do if the stress loss limit is 20M USD per cpty, can you work out the room in each cpties then total it?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_SF6g4xpB0OpXdUISjIvh77W2): maxloss *****
Arguments: 
{}
************************************************************************
***** Suggested tool call (call_8oTRk9XQD1Ie80aCSHJfYfvu): cpty_exp *****
Arguments: 
{}
*************************************************************************
***** Suggested tool call (call_OmU0XankfCC1qHujRX2STzoJ): scenarios *****
Arguments: 
{}
**************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION maxloss...

>>>>>>>> EXECUTING FUNCTION cpty_exp...

>>>>>>>> EXECUTING FUNCTION sce

/var/folders/jv/hd_kl00j2hxfm6sn1phxsr1m0000gn/T/ipykernel_35324/412691332.py:296: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  marked_to_market_df = mtm_df.groupby(['Counterparty', 'Scenario']).sum()


Assistant (to User):

To determine how much more room we have to buy in USD/GBP notional, given a stress loss limit of 20M USD per counterparty, we need to follow several steps:

1. **Extract Stress Scenarios:**
   - Brexit: GBP = -0.1
   - US-China Trade War: GBP = -0.05
   - COVID-19: GBP = 0.1

2. **Extract Counterparty Exposures in GBP:**
   - Morgan Stanley: 173,819,591 USD
   - Goldman Sachs: 268,706,852 USD
   - Deutsche Bank: 192,401,812 USD
   - JP Morgan: 112,039,152 USD
   - BNY: 250,563,230 USD

3. **Find Maximum Losses from Stress Scenarios:**
   - Morgan Stanley: Brexit: 99,157.565, US-China Trade War: -10,841,500.450, COVID-19: 16,474,304.190 (max absolute value = 16,474,304.190)
   - Goldman Sachs: Brexit: 5,328,591.700, US-China Trade War: 3,786,505.650, COVID-19: -6,688,186.960 (max absolute value = 6,688,186.960)
   - Deutsche Bank: Brexit: -6,069,641.100, US-China Trade War: 702,925.950, COVID-19: -2,175,289.630 (max absolute value = 6,069,641.100)
   - JP Morgan: B

In [229]:
print(chatresult.summary)

To determine how much more notional can be traded for USD/GBP given a stress loss limit of $20M USD per counterparty, we need to calculate the current stress loss for each counterparty and determine the remaining capacity before reaching the limit.

### Stress Scenarios Impact for USD/GBP
- Brexit: -0.1
- US-China Trade War: -0.05
- COVID-19: 0.1

We will calculate the current stress loss under these scenarios for USD/GBP and then figure out how much more notional can be added before hitting the $20M limit.

1. **Morgan Stanley:** $173,819,591
2. **Goldman Sachs:** $268,706,852
3. **Deutsche Bank:** $192,401,812
4. **JP Morgan:** $112,039,152
5. **BNY:** $250,563,230

Using the worst-case scenario for each (maximum negative impact):
- Brexit: -0.1 for GBP

Stress Loss Calculation:
\[ \text{Stress Loss} = \text{Notional} \times \text{Stress Scenario Impact} \]

We will now calculate the stress losses and remaining capacities for each counterparty:

### Morgan Stanley
\[ \text{Stress Los